# NTLK Quetes 4  Bag of words

* 1 Importe cet ensemble de données de tweets dans un DataFrame.
* 2 Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?
* 3 Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75.
* 4 Crée un modèle vectorizer avec scikit-learn en utilisant la méthode Countvectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.
* 5 Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966 pour le test d'entraînement, et * 0.877 pour l'ensemble de test.
*  6 Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

## imports

In [1]:
import os
import io
import nltk
import gzip
import spacy
import string
import random
import secrets
import datetime
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import svm
import plotly.io as pio
from sklearn import tree
from typing import Counter
import plotly.express as px
from fuzzywuzzy import fuzz
from textblob import TextBlob
from joblib import dump, load
from bs4 import BeautifulSoup
import category_encoders as ce
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import plotly.graph_objects as go
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
from nltk.stem import SnowballStemmer
from sklearn.decomposition import PCA
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score
from scipy.spatial.distance import pdist, squareform
from textblob_fr import PatternTagger, PatternAnalyzer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from flask import Flask, request, render_template, session, url_for, redirect
from sklearn.preprocessing import (MaxAbsScaler, MinMaxScaler, Normalizer, PowerTransformer, QuantileTransformer, RobustScaler, StandardScaler)
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, TargetEncoder



c:\Users\romar\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Conseils
* Train test split
* Fit et transform du Vectorizer sur le train set
* Fit et score du classifier sur le train set
* Transform (sans fit !) du Vectorizer sur le test set
* Predict et score du classifier sur le test set

### Paramètres du CountVectorizer

A l'initialisation du CountVectorizer, tu peux spécifier quelques paramètres très intéressants. Citons notamment :

* lowercase : permet de convertir tout le texte en minuscule
* stop_words : permet de spécifier une liste de stopwords, qui ne généreront donc pas de colonnes dédiées
* ngram_range : permet de spécifier si des bigrammes ou n-grammes doivent être pris en compte
* max_features : limite le nombre de mots maximum, en ne prenant que les mots les plus fréquents


In [17]:
# fichier_to_test = "./le_bonheur.txt"
# print(f"\n fichier_to_test :\n{fichier_to_test} \n")

# # Lire le fichier
# with open(fichier_to_test, "r", encoding="utf-8") as file:
#     texte = file.read()

# # Tokenisation par phrase, liste les phrases dans le texte
# token_phrases = nltk.sent_tokenize(texte)
# print(f"\n token_phrases:\n{token_phrases} \n")

# Charger le DataFrame à partir du fichier CSV
df = pd.read_csv("tweets_train.csv")
# on ne garde pas les lignes \n\n
# df = df['sentiment']
# tokens = df['tokens'].tolist()

# # transformation en matrice creuse
# vectorizer = CountVectorizer()
# matrice_creuse=vectorizer.fit_transform(df['sentiment'])
# print(f"\n matrice_creuse:\n{matrice_creuse.shape[1]} \n")
# # afficher la liste des noms de colonnes
# liste_colonne_matrice_creuse=vectorizer.get_feature_names_out()
# print(f"\n liste_colonne_matrice_creuse:\n{liste_colonne_matrice_creuse} \n")



 matrice_creuse:
3 


 liste_colonne_matrice_creuse:
['negative' 'neutral' 'positive'] 



1. Filtrer les Tweets Positifs et Négatifs
['negative' 'neutral' 'positive'] 


In [30]:
# Charger le DataFrame à partir du fichier CSV
df = pd.read_csv("tweets_train.csv")
print(f"\n df.info():\n{df.info()} \n")
# on ne garde pas les lignes \n\n
#  Filtre les tewets positifs et negatifs
df_filtered = df[df['sentiment'].isin(['positive', 'negative'])]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB

 df.info():
None 



2. Calculer le Pourcentage de Tweets Positifs/Négatifs

In [32]:
pourcentage_positif = (df_filtered['sentiment'] == 'positive').mean()
pourcentage_negatif = (df_filtered['sentiment'] == 'negative').mean()
print(f"\n pourcentage_positif:\n{pourcentage_positif:.2f} \n")
print(f"\n pourcentage_negatif:\n{pourcentage_negatif:.2f} \n")



 pourcentage_positif:
0.52 


 pourcentage_negatif:
0.48 



3. Préparation des Données pour le Modèle

In [33]:
X = df_filtered['text']
y = df_filtered['sentiment']


4. Split Train/Test

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)
# Vérifier la dimension de X_test_CV
X_test_CV.shape
print(f"\n X_test_CV.shape:\n{X_test_CV.shape} \n")
print(f"\n X_test_CV.size:\n{X_test_CV.size} \n")



 X_test_CV.shape:
(4091, 15806) 


 X_test_CV.size:
44633 



5. Vectorisation
 Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_CV = vectorizer.fit_transform(X_train)
X_test_CV = vectorizer.transform(X_test)

print(f"\n X_train_CV:\n{X_train_CV} \n")
print(f"\n X_test_CV:\n{X_test_CV} \n")
print(f"\n X_train_CV:\n{X_train_CV.toarray()} \n")
print(f"\n X_test_CV:\n{X_test_CV.toarray()} \n")



 X_train_CV:
(12272, 15806) 


 X_test_CV:
  (0, 1462)	1
  (0, 7575)	1
  (0, 7883)	1
  (0, 9812)	1
  (0, 12524)	1
  (0, 13294)	1
  (1, 1230)	1
  (1, 2169)	1
  (1, 2227)	1
  (1, 4414)	1
  (1, 4636)	1
  (1, 4645)	1
  (1, 6158)	1
  (1, 6301)	1
  (1, 7591)	1
  (1, 9910)	1
  (1, 10078)	1
  (1, 10617)	1
  (1, 11964)	1
  (1, 12189)	1
  (1, 12838)	1
  (1, 13163)	1
  (1, 13829)	1
  (1, 15209)	1
  (1, 15355)	1
  :	:
  (4087, 12987)	1
  (4087, 13829)	1
  (4087, 13837)	1
  (4087, 15212)	1
  (4088, 12189)	1
  (4088, 12449)	1
  (4088, 13829)	1
  (4088, 14042)	1
  (4088, 15009)	1
  (4088, 15030)	1
  (4088, 15192)	1
  (4089, 1902)	1
  (4089, 7196)	1
  (4089, 9537)	1
  (4090, 2108)	1
  (4090, 2126)	1
  (4090, 2473)	1
  (4090, 7362)	1
  (4090, 8279)	1
  (4090, 9509)	1
  (4090, 9744)	1
  (4090, 12810)	1
  (4090, 13812)	1
  (4090, 15355)	1
  (4090, 15684)	1 


 X_train_CV:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 


 X_test_CV


* 5 Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966 pour le test d'entraînement, et * 0.877 pour l'ensemble de test.
*  6 Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

6. Entraînement d'une Régression Logistique

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)
model.fit(X_train_CV, y_train)

# Évaluer le modèle
accuracy_train = model.score(X_train_CV, y_train)
accuracy_test = model.score(X_test_CV, y_test)

print(f"\n accuracy_train:{accuracy_train:.3f} ")
print(f"\n accuracy_test:{accuracy_test:.3f} ")



 accuracy_train:0.966 

 accuracy_test:0.877 


Étape Bonus : Affichage des Tweets Mal Prédits

In [51]:
y_pred = model.predict(X_test_CV)
mal_predits = X_test[(y_pred != y_test)]
print(mal_predits.head(10))
# performance globale du modele avec f1
# utiliser recall pour avoir le % de positifs bien predits
#   df.niveau1.value_counts()
#  faire un tfidf


2670     HollowbabesHere comes the utter shite #bgt <I ...
18731     SUFFICATION NO BREATHING. It`s okay. There`ll...
12054    i wanna vote for Miley Cyrus for the mtv movie...
21823    I love music so much that i`ve gone through pa...
18464    I can only message those who message me, if we...
2975     wish I could feel no pain (8)  but it`s ok, at...
3921                        so glad i`m not at uni anymore
5198      You`re not here. I hope you`re still resting....
467        you`re missing out, bb! i`m such a cereal nu...
15215     have an amazing time with your mommas tomorro...
Name: text, dtype: object
